In [4]:

def get_key(val, my_dict):
    for key, value in my_dict.items():
         if val == value:
             return key
    return "key doesn't exist"

In [29]:
#This just sets up our basic objects
class State:
    def __init__(self, name, transition_vector, observation_liklihoods):
        self.name = name
        # A dict that contains the name state it can transtion to, and the likelihood that you do it
        self.a = transition_vector
        # A dict that contains all possibles outputs and their liklihood
        self.b = observation_liklihoods


class HMM:
    def __init__(self, states, initial_probabilities):
        # A list of all possible states
        self.q = states
        # The likelihood of starting in each state (dict)
        self.pie = initial_probabilities
        # Initialization of alpha variable with dummy data
        self.alpha = {state.name:1 for state in states}

    def likelihood(self, observation_sequence):
        '''
        Given a sequence of observations find the likelihood that they occured with the given model
        '''
        #Initialization
        self.initial_alpha(observation_sequence[0])
        #Recursion
        for observation in observation_sequence[1:]:
            self.calculate_alpha(observation)
        #Termination
        return sum(self.alpha.values())

    def decode(self, observation_sequence):
        self.initial_alpha(observation_sequence[0])
        back_trace = []
        path = []
        for observation in observation_sequence[1:]:
            back_trace.append(self.viterbi(observation))
        print(get_key(max(self.alpha.values()), self.alpha))
        #Get the most likly final state
        path.append(get_key(max(self.alpha.values()), self.alpha))
        #Walk backwards graphing the most likli previous state from this one
        print(back_trace)
        for trace in back_trace[::-1]:
            #Add to the final path the previous state that was the most liklione to get here
            print(path[-1])
            path.append(trace[path[-1]])
        return path[::-1]
    
    def initial_alpha(self, observation):
        for state in self.q: 
            self.alpha[state.name] = self.pie[state.name] * state.b[observation]   


    def calculate_alpha(self, observation):
		#Store the previous alpha values for calculation later
        previous_alpha = self.alpha.copy()
		#Loop through every state you going to ->i
        for state_i in self.q: 
            transition_obv = []
			#Loop through every state you are coming from j->
            for state_j in self.q: 
                #Calculate the liklihood of seeing the observation given the state i
                transition_obv.append(state_j.a[state_i.name] * state_i.b[observation] * previous_alpha[state_j.name])
                #inter_step = state_j.a[state_i.name] * state_i.b[observation]
                print("Going from state {1} -> {0}: {2} X emission: {3} = {5} X previous_alpha {4}"
                .format(state_i.name, state_j.name, state_j.a[state_i.name], state_i.b[observation], previous_alpha[state_j.name], inter_step))
            self.alpha[state_i.name] = sum(transition_obv)
    
    def viterbi(self, observation):
        previous_alpha = self.alpha.copy()
        back_trace = {}
        for state_i in self.q: 
            transition_obv = {}
            for state_j in self.q: 
                #Calculate the liklihood of seeing the observation given the state i
                transition_obv[state_j.name] = (state_j.a[state_i.name] * state_i.b[observation] * previous_alpha[state_j.name])
                inter_step = state_j.a[state_i.name] * state_i.b[observation]
                print("Going from state {1} -> {0}: {2} X emission: {3} = {5} X previous_alpha {4}"
                .format(state_i.name, state_j.name, state_j.a[state_i.name], state_i.b[observation], previous_alpha[state_j.name], inter_step))
            #print("Total = {0}".format(sum(transition_obv)))
            self.alpha[state_i.name] = max(transition_obv.values())
            back_trace[state_i.name] = get_key(self.alpha[state_i.name], transition_obv)
        return back_trace





In [30]:
states = []
hot_transition = {'Hot':.6, 'Cold':.4}
hot_emission = {1:.2, 2:.4, 3:.4}
cold_transition = {'Hot':.5, 'Cold':.5}
cold_emission = {1:.5, 2:.4, 3:.1}


hot = State('Hot', hot_transition, hot_emission)
cold = State('Cold', cold_transition, cold_emission)
hmm = HMM([hot, cold], {"Hot":.8, "Cold":.2})


In [33]:

#hmm.likelihood([1,2,2])
hmm.decode([3,1,3,3,2,3,1,2,3,3])

Going from state Hot -> Hot: 0.6 X emission: 0.2 = 0.12 X previous_alpha 0.32000000000000006
Going from state Cold -> Hot: 0.5 X emission: 0.2 = 0.1 X previous_alpha 0.020000000000000004
Going from state Hot -> Cold: 0.4 X emission: 0.5 = 0.2 X previous_alpha 0.32000000000000006
Going from state Cold -> Cold: 0.5 X emission: 0.5 = 0.25 X previous_alpha 0.020000000000000004
Going from state Hot -> Hot: 0.6 X emission: 0.4 = 0.24 X previous_alpha 0.038400000000000004
Going from state Cold -> Hot: 0.5 X emission: 0.4 = 0.2 X previous_alpha 0.06400000000000002
Going from state Hot -> Cold: 0.4 X emission: 0.1 = 0.04000000000000001 X previous_alpha 0.038400000000000004
Going from state Cold -> Cold: 0.5 X emission: 0.1 = 0.05 X previous_alpha 0.06400000000000002
Going from state Hot -> Hot: 0.6 X emission: 0.4 = 0.24 X previous_alpha 0.012800000000000004
Going from state Cold -> Hot: 0.5 X emission: 0.4 = 0.2 X previous_alpha 0.003200000000000001
Going from state Hot -> Cold: 0.4 X emission

['Hot', 'Cold', 'Hot', 'Hot', 'Hot', 'Hot', 'Cold', 'Hot', 'Hot', 'Hot']

We know the sum of all probabilites after any given times step T should be 1

In [17]:
likelihoods = []
#Get all possible combinations of outputs for 3 times steps
min = 1
min_val = []
for i in range(1,4):
    for j in range(1,4):
        for k in range(1,4):
            a = hmm.likelihood([i,j,k])
            if (a < min):
                min = a
                min_val = [i,j,k]
            likelihoods.append(hmm.likelihood([i,j,k]))
           
print(sum(likelihoods))
print(min)
print(min_val)

1.0000000000000002
0.019202000000000004
[1, 3, 3]


'hello world'

0.6